# CHAPTER 2 - The Tools of the Trade in Quantum Computing - PennyLane Code

*Note*: You may skip the following three cells if you have alredy installed the right versions of all the libraries mentioned in *Appendix D*. This will likely NOT be the case if you are running this notebook on a cloud service such as Google Colab.



In [ ]:
pip install pennylane==0.26

In [ ]:
pip install qiskit==0.39.2

In [ ]:
pip install pennylane-qiskit==0.24.0

In [1]:
import pennylane as qml

In [2]:
dev = qml.device('default.qubit', wires = 2)

In [11]:
def qc():
    qml.PauliX(wires = 0)
    qml.Hadamard(wires = 0)
    return qml.state()

In [12]:
qcirc = qml.QNode(qc, dev) # Assemble the circuit & the device.
qcirc() # Run it!

tensor([ 0.70710678+0.j, -0.70710678+0.j], requires_grad=True)

In [13]:
@qml.qnode(dev) # We add this decorator to use the device dev.
def qcirc():
    qml.PauliX(wires = 0)
    qml.Hadamard(wires = 0)
    return qml.state()
    
# Now qcirc is already a QNode. We can just run it!
qcirc()

tensor([ 0.70710678+0.j, -0.70710678+0.j], requires_grad=True)

In [14]:
dev = qml.device('default.qubit', wires = 1)
@qml.qnode(dev)
def qcirc(theta):
    qml.RX(theta, wires = 0)
    return qml.state()

In [15]:
print(qml.draw(qcirc)(theta = 2))

0: ──RX(2.00)─┤  State


In [16]:
dev = qml.device('default.qubit', wires = 3)

# Get probabilities
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires = 1)
    return qml.probs(wires = [1, 2]) # Only the last 2 wires.
prob = qcirc()
print("Probs. wires [1, 2] with H in wire 1:", prob)

# Get a sample, not having specified shots in the device.
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires = 0)
    return qml.sample(wires = 0) # Only the first wire.
s1 = qcirc(shots = 4) # We specify the shots here.
print("Sample 1 after H:", s1)

# Get a sample with shots in the device.
dev = qml.device('default.qubit', wires = 2, shots = 4)
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires=0)
    return qml.sample() # Will sample all wires.
s2 = qcirc()
print("Sample 2 after H x I:", s2)

Probs. wires [1, 2] with H in wire 1: [0.5 0.  0.5 0. ]
Sample 1 after H: [0 1 0 1]
Sample 2 after H x I: [[0 0]
 [0 0]
 [1 0]
 [0 0]]


In [17]:
dev = qml.device('qiskit.aer', wires = 2)
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires = 0)
    return qml.probs(wires = 0)
s = qcirc()
print("The probabilities are", s)

The probabilities are [0.5078125 0.4921875]


*Note*: In the following cell, you need to replace "1234" with your actual IBM token. Refer to *Appendix D* in the book for instructions on how to create an account and get your token. Be very careful not to disclose your token to anyone!

In [ ]:
ibm_token = "1234"

In [ ]:
# Import the modules.
from qiskit import *
from qiskit.providers.ibmq import *

# Save our token if we haven't already.
IBMQ.save_account(ibm_token)

# Load the account and get the name of the least busy backend.
prov = IBMQ.load_account()
bck = least_busy(prov.backends(simulator = False)).name()

# Invoke the PennyLane IBMQ device.
dev = qml.device('qiskit.ibmq', wires = 1,
    backend = bck, provider = prov)

# Send a circuit and get some results!
@qml.qnode(dev)
def qcirc():
    qml.Hadamard(wires = 0)
    return qml.probs(wires = 0)
print(qcirc())